In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

# Functions

In [2]:
from dadata import Dadata

In [3]:
login = 'ow-admin@tochka.com'
password = 'Tochka-geo61'
token = 'ed714e4b6c4306abf7b9813c3447963b693e1370'

In [4]:
dadata = Dadata(token)

In [9]:
dadata.find_by_id("address", '0200000101500')

[{'value': 'г Уфа, деревня Жилино',
  'unrestricted_value': '450902, Респ Башкортостан, г Уфа, Октябрьский р-н, деревня Жилино',
  'data': {'postal_code': '450902',
   'country': 'Россия',
   'country_iso_code': 'RU',
   'federal_district': 'Приволжский',
   'region_fias_id': '6f2cbfd8-692a-4ee4-9b16-067210bde3fc',
   'region_kladr_id': '0200000000000',
   'region_iso_code': 'RU-BA',
   'region_with_type': 'Респ Башкортостан',
   'region_type': 'Респ',
   'region_type_full': 'республика',
   'region': 'Башкортостан',
   'area_fias_id': None,
   'area_kladr_id': None,
   'area_with_type': None,
   'area_type': None,
   'area_type_full': None,
   'area': None,
   'city_fias_id': '7339e834-2cb4-4734-a4c7-1fca2c66e562',
   'city_kladr_id': '0200000100000',
   'city_with_type': 'г Уфа',
   'city_type': 'г',
   'city_type_full': 'город',
   'city': 'Уфа',
   'city_area': None,
   'city_district_fias_id': '67123c23-a198-422c-8c74-13c8ca57ba09',
   'city_district_kladr_id': None,
   'city_dist

# Data

## Region

In [6]:
# reg = 66 # Екатеринбург
# reg = 52 # Нижний Новгород
# reg = 72 # Тюмень
# reg = 45 # Курган
# reg = 50 # Московская
# reg = '23' # Краснодар
reg = 70 # Томск

## Load KLADR

In [8]:
df = pd.read_csv(f'/home/tochka/Documents/Other/Work/Geo/EGRUL/kladr/df_kladr_{reg}.csv.gz', compression='gzip', dtype={'kladr_11':str})

In [9]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   kladr_11  317 non-null    object
dtypes: object(1)
memory usage: 2.6+ KB


/tmp/ipykernel_39569/2612161665.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(verbose=True, null_counts=True)


In [10]:
df.head()

,kladr_11
0,70000001000
1,70013000020
2,70000004000
3,70009001000
4,70017000012


In [11]:
df.groupby(df.kladr_11.map(len)).size()

kladr_11
11    317
dtype: int64

In [12]:
df['kladr'] = df['kladr_11'] + '00'

In [13]:
df.head()

,kladr_11,kladr
0,70000001000,7000000100000
1,70013000020,7001300002000
2,70000004000,7000000400000
3,70009001000,7000900100000
4,70017000012,7001700001200


## Coordinates KLADR

In [14]:
kladr_list = df['kladr']

In [15]:
kladr_list

0      7000000100000
1      7001300002000
2      7000000400000
3      7000900100000
4      7001700001200
           ...      
312    7000900001500
313    7000300000700
314    7000000102000
315    7001700008400
316    7001700001300
Name: kladr, Length: 317, dtype: object

In [16]:
def kladr_to_coodr(kladr):
    response = dadata.find_by_id("address", kladr)
    address = response[0]['unrestricted_value']
    latitude = response[0]['data']['geo_lat']
    longitude = response[0]['data']['geo_lon']
    return address, latitude, longitude

In [17]:
addresses = []
latitudes = []
longitudes = []

for kladr in tqdm(kladr_list):
    try:
        address, latitude, longitude = kladr_to_coodr(kladr)
        addresses.append(address)
        latitudes.append(latitude)
        longitudes.append(longitude)
        # print(address, latitude, longitude)
    except Exception as e:
        addresses.append(None)
        latitudes.append(None)
        longitudes.append(None)
        # print(e)

100%|█████████████████████████████████████████| 317/317 [01:26<00:00,  3.66it/s]


In [18]:
len(addresses)

317

In [19]:
df['address'] = addresses

In [20]:
df['latitude'] = latitudes

In [21]:
df['longitude'] = longitudes

In [22]:
df.shape

(317, 5)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   kladr_11   317 non-null    object
 1   kladr      317 non-null    object
 2   address    312 non-null    object
 3   latitude   310 non-null    object
 4   longitude  310 non-null    object
dtypes: object(5)
memory usage: 12.5+ KB


In [24]:
df.head()

,kladr_11,kladr,address,latitude,longitude
0,70000001000,7000000100000,"634000, Томская обл, г Томск",56.48468,84.948197
1,70013000020,7001300002000,"636953, Томская обл, Первомайский р-н, село Ку...",56.949172,86.465613
2,70000004000,7000000400000,"636780, Томская обл, г Стрежевой",60.732801,77.604047
3,70009001000,7000900100000,"636460, Томская обл, Колпашевский р-н, г Колпа...",58.311384,82.902679
4,70017000012,7001700001200,"634582, Томская обл, Томский р-н, село Алексан...",56.74143,85.388326


In [25]:
df.to_csv(f'/home/tochka/Documents/Other/Work/Geo/EGRUL/kladr/df_kladr_{reg}_upd.csv.gz', sep=';', index=False, compression='gzip')

## Load FIAS

In [26]:
df_fias = pd.read_csv(f'/home/tochka/Documents/Other/Work/Geo/EGRUL/fias/df_fias_{reg}.csv.gz', compression='gzip')

In [27]:
df_fias.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2149 entries, 0 to 2148
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fias_code  2149 non-null   object
dtypes: object(1)
memory usage: 16.9+ KB


/tmp/ipykernel_39569/1671388246.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_fias.info(verbose=True, null_counts=True)


In [28]:
df_fias.head()

,fias_code
0,83da1256-a96d-47c7-9dce-78742c0fa454
1,0a4d3c20-a7e0-4cd9-b17e-9022dc79a663
2,e940a45e-b29f-451e-a0f3-1974df8e22ae
3,16ba864e-241a-44a5-ac76-d30d591a9280
4,683d98a6-5eee-46d8-aba6-030af4d2b060


## Coordinates FIAS

In [29]:
fias_list = df_fias['fias_code']

In [30]:
fias_list

0       83da1256-a96d-47c7-9dce-78742c0fa454
1       0a4d3c20-a7e0-4cd9-b17e-9022dc79a663
2       e940a45e-b29f-451e-a0f3-1974df8e22ae
3       16ba864e-241a-44a5-ac76-d30d591a9280
4       683d98a6-5eee-46d8-aba6-030af4d2b060
                        ...                 
2144    05ce32f5-e167-44d9-b973-2dc681222e59
2145    1b14b308-b84b-42cf-8bc6-fe4717b3dca6
2146    8283393c-1b6e-40f7-8ae2-a4ca386b95bc
2147    768dd377-e84c-4a09-9f33-3fe896fd56b5
2148    21f87f43-c2d8-4f0f-b5f0-87a6c7dd15da
Name: fias_code, Length: 2149, dtype: object

In [31]:
def fias_to_coodr(fias):
    response = dadata.find_by_id("address", str(fias))
    address = response[0]['unrestricted_value']
    latitude = response[0]['data']['geo_lat']
    longitude = response[0]['data']['geo_lon']
    return address, latitude, longitude

In [32]:
addresses_fias = []
latitudes_fias = []
longitudes_fias = []

for kladr in tqdm(fias_list[:10]):
    try:
        address, latitude, longitude = fias_to_coodr(kladr)
        addresses_fias.append(address)
        latitudes_fias.append(latitude)
        longitudes_fias.append(longitude)
        # print(address, latitude, longitude)
    except Exception as e:
        addresses_fias.append(None)
        latitudes_fias.append(None)
        longitudes_fias.append(None)
        # print(e)

100%|███████████████████████████████████████████| 15/15 [00:05<00:00,  2.65it/s]


In [33]:
len(addresses_fias)

15

In [34]:
addresses_fias

['634049, Томская обл, г Томск, Иркутский тракт, д 13/1',
 '634020, Томская обл, г Томск, ул Сибирская, д 80',
 '634507, Томская обл, Томский р-н, поселок Зональная Станция, ул Совхозная, д 2а',
 '636844, Томская обл, Асиновский р-н, г Асино, ул имени Ленина, д 172/1',
 '634009, Томская обл, г Томск, ул Бердская',
 '634507, Томская обл, Томский р-н, поселок Зональная Станция, ул 40 лет Победы, д 2',
 '636842, Томская обл, Асиновский р-н, г Асино, ул Степная, д 1',
 '634061, Томская обл, г Томск, ул Киевская, д 1',
 '634009, Томская обл, г Томск, Совпартшкольный пер, д 13',
 '634032, Томская обл, г Томск, ул Бирюкова, д 2',
 '634015, Томская обл, г Томск, 1-й Вилюйский проезд, д 8 стр 5',
 '634538, Томская обл, Томский р-н, село Корнилово, ул Гагарина, д 29а',
 '634024, Томская обл, г Томск, ул Усть-Керепеть',
 '634050, Томская обл, г Томск, ул Татарская',
 '634059, Томская обл, г Томск, ул Ференца Мюнниха, д 34']

In [35]:
latitudes_fias

['56.50067',
 '56.482672',
 '56.430987',
 '56.999274',
 '56.511168',
 '56.426094',
 '56.99419',
 '56.48375',
 '56.491911',
 '56.5051404',
 '56.5250686',
 '56.495509',
 '56.525594',
 '56.47796',
 '56.5184187']

In [36]:
longitudes_fias

['85.00571',
 '84.993858',
 '85.008581',
 '86.141671',
 '84.942942',
 '85.02207',
 '86.1854',
 '84.9756',
 '84.949778',
 '85.0350428',
 '85.0047201',
 '85.170108',
 '84.930877',
 '84.944909',
 '84.984918']

In [37]:
addresses_fias = []
latitudes_fias = []
longitudes_fias = []

for kladr in tqdm(fias_list):
    try:
        address, latitude, longitude = fias_to_coodr(kladr)
        addresses_fias.append(address)
        latitudes_fias.append(latitude)
        longitudes_fias.append(longitude)
        # print(address, latitude, longitude)
    except Exception as e:
        addresses_fias.append(None)
        latitudes_fias.append(None)
        longitudes_fias.append(None)
        # print(e)

100%|███████████████████████████████████████| 2149/2149 [12:01<00:00,  2.98it/s]


In [38]:
len(addresses_fias)

2149

In [39]:
df_fias['address'] = addresses_fias

In [40]:
df_fias['latitude'] = latitudes_fias

In [41]:
df_fias['longitude'] = longitudes_fias

In [42]:
df_fias.shape

(2149, 4)

In [43]:
df_fias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2149 entries, 0 to 2148
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fias_code  2149 non-null   object
 1   address    2149 non-null   object
 2   latitude   2145 non-null   object
 3   longitude  2145 non-null   object
dtypes: object(4)
memory usage: 67.3+ KB


In [44]:
df_fias.head()

,fias_code,address,latitude,longitude
0,83da1256-a96d-47c7-9dce-78742c0fa454,"634049, Томская обл, г Томск, Иркутский тракт,...",56.50067,85.00571
1,0a4d3c20-a7e0-4cd9-b17e-9022dc79a663,"634020, Томская обл, г Томск, ул Сибирская, д 80",56.482672,84.993858
2,e940a45e-b29f-451e-a0f3-1974df8e22ae,"634507, Томская обл, Томский р-н, поселок Зона...",56.430987,85.008581
3,16ba864e-241a-44a5-ac76-d30d591a9280,"636844, Томская обл, Асиновский р-н, г Асино, ...",56.999274,86.141671
4,683d98a6-5eee-46d8-aba6-030af4d2b060,"634009, Томская обл, г Томск, ул Бердская",56.511168,84.942942


In [45]:
df_fias.to_csv(f'/home/tochka/Documents/Other/Work/Geo/EGRUL/fias/df_fias_{reg}_upd.csv.gz', sep=';', index=False, compression='gzip')